In [45]:
from absl import logging

import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import string
import operator
from IPython.display import HTML

In [46]:
import pickle

In [47]:
from Folse.flask.pythonModules import preProcess

In [48]:
df = pd.read_csv('fo.csv', encoding='ISO-8859-1')
fund_objectives = df['Objective'].values

In [49]:
data_cleaned = preProcess.pre_process_sent(fund_objectives)
#print(data_cleaned)

In [50]:
# ELMO expects the input to be sentences rather than word tokens
# therefore, join back the cleaned tokens to sentences
data_cleaned_sentences = []
for arr in data_cleaned:
    data_cleaned_sentences.append(" ".join(arr))
#print(data_cleaned_sentences)

In [51]:
print(len(data_cleaned_sentences))
print(data_cleaned_sentences[0:10])

771
['maximize total returns current income capital appreciation investing primarily financial credit sector globally including banks insurance finance companies focusing subordinated debt securities financial credit sector including contingent convertible bonds cocos preferred shares may rated investment grade investment grade', 'high total investment return investing global portfolio shortduration investment grade fixedincome securities hedging nonus dollardenominated positions us dollars minimize risks currency fluctuations drawing large global team evaluate topdown factors influencing markets well bottomup factors affecting bond prices', 'longterm capital growth investing stocks across eurozone offer compelling longterm return potential attractive valuations taking longterm perspective order capitalize shortterm market inefficiencies created investors overreaction macroeconomic industry companyspecific concerns employing disciplined bottomup approach combines fundamental research p

In [52]:
url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url)

In [53]:
def elmo_vectors(x):
  embeddings = embed(x, signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [54]:
#embeddings = embed(
#    fund_objectives[:400],
#    signature="default",
#    as_dict=True)["default"]

In [55]:
# create a generator so that the ELMO vectors are generated in small chunks of 100
# this is essential otherwise we will face OUT of MEMORY errors
list_data_cleaned_sentences = [data_cleaned_sentences[i:i+100] for i in range(0,len(data_cleaned_sentences),100)]

In [56]:
elmo_doc_vectors = [elmo_vectors(x) for x in list_data_cleaned_sentences]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [57]:
elmo_doc_vectors_concat = np.concatenate(elmo_doc_vectors, axis = 0)

In [58]:
pickle_out = open("elmo_vectors_03122019.pickle","wb")
pickle.dump(elmo_doc_vectors_concat, pickle_out)
pickle_out.close()

In [59]:
# load the elmo vectors from the disk for re-use
pickle_in = open("elmo_vectors_03122019.pickle", "rb")
elmo_doc_vectors_load = pickle.load(pickle_in)

In [60]:
#%%time
#with tf.Session() as sess:
#  sess.run(tf.global_variables_initializer())
#  sess.run(tf.tables_initializer())
#  x = sess.run(elmo_doc_vectors_load)

In [74]:
#@title Sementic search
#@markdown Enter a set of words to find matching sentences. 'results_returned' can beused to modify the number of matching sentences retured. To view the code behind this cell, use the menu in the top right to unhide...
#search_string = "The fund's investment objective is to achieve long-term growth of capital and income" #@param {type:"string"}
search_string = 'i want to invest in funds with capital appreciation and income growth'
search_string_pre_process = preProcess.pre_process_sent(search_string)[0]
search_string_pre_process = " ".join(search_string_pre_process)

results_returned = "5" #@param [1, 2, 3]

from sklearn.metrics.pairwise import cosine_similarity


embeddings2 = embed(
    [search_string_pre_process],
    signature="default",
    as_dict=True)["default"]

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  sess.run(tf.tables_initializer())
  search_vect = sess.run(embeddings2)
  

#cosine_similarities = pd.Series(cosine_similarity(search_vect, x).flatten())
cosine_similarities = pd.Series(cosine_similarity(search_vect, elmo_doc_vectors_load).flatten())
output =""
for i,j in cosine_similarities.nlargest(int(results_returned)).iteritems():
  output +='<p style="font-family:verdana; font-size:110%;"> '
  output += str(round(j,5)) + ", "
  for s in fund_objectives[i].split():
    if s.lower() in search_string:
      output += " <b>"+str(s)+"</b>"
    else:
      output += " "+str(s)
  output += "</p><hr>"
    
output = '<h3>Results:</h3>'+output
display(HTML(output))
#   print(sentences[i])
#   print('\n')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [76]:
print(cosine_similarities.nlargest(5))

415    0.780797
416    0.780797
476    0.765640
477    0.765640
605    0.756741
dtype: float32


# Visualizations

In [62]:
from sklearn.decomposition import PCA

pca = PCA(n_components=50)
y = pca.fit_transform(elmo_doc_vectors_load)

from sklearn.manifold import TSNE

y = TSNE(n_components=2).fit_transform(y)

In [63]:
sentences = fund_objectives

In [64]:
#import plotly.plotly as py
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)


data = [
    go.Scatter(
        x=[i[0] for i in y],
        y=[i[1] for i in y],
        mode='markers',
        text=[i for i in sentences],
    marker=dict(
        size=16,
        color = [len(i) for i in sentences], #set color equal to a variable
        opacity= 0.8,
        colorscale='Viridis',
        showscale=False
    )
    )
]
layout = go.Layout()
layout = dict(
              yaxis = dict(zeroline = False),
              xaxis = dict(zeroline = False)
             )
fig = go.Figure(data=data, layout=layout)
file = plot(fig, filename='Sentence encode.html')

In [67]:
search_string = 'short duration investment grade fixed income securities hedging non us'
search_string_pre_process = preProcess.pre_process_sent(search_string)[0]

In [69]:
search_string_pre_process = " ".join(search_string_pre_process)

In [70]:
search_string_pre_process

'short duration investment grade fixed income securities hedging non us'